# Stage 2: The Study Session (Fine-Tuning)

Welcome to the training ground! This notebook will teach Llama-3 how to label your bodywash reviews.

### Pre-requisites
1.  Make sure you are connected to a GPU (**Runtime > Change runtime type > T4 GPU**).
2.  Make sure you uploaded `dataset_train.jsonl` to the files area on the left.

In [ ]:
%%capture
# 1. Install the Training Tools (Unsloth)
# This installs the specific libraries needed to train Llama-3 efficiently.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
# 2. Load the Student (The Base Model)
from unsloth import FastLanguageModel
import torch

print("Loading Llama-3 model... (This may take a minute)")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-instruct-bnb-4bit", # We use the 4-bit version to fit in memory
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
print("Model loaded!")

In [ ]:
# 3. Add the 'Expert Layer' (LoRA Adapters)
# We don't train the whole brain, just these specific parts.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [ ]:
# 4. Load the Flashcards (Data)
from datasets import load_dataset

# Format function to look like a chat
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Load data
try:
    dataset = load_dataset("json", data_files="dataset_train.jsonl", split="train")
    dataset = dataset.map(formatting_prompts_func, batched = True)
    print("Data loaded successfully!")
except Exception as e:
    print("ERROR: Could not load dataset_train.jsonl. Did you upload it?")
    print(e)

In [ ]:
# 5. Start Studying (Training)
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Quick run (approx 10-15 mins). Increase to 300 for better results if you have time.
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

print("Starting training...")
trainer.train()
print("Training Complete!")

In [ ]:
# 6. Quick Verification
# Let's ask it a test question to see if it learned.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Identify the 'Level 1 Factors' (attributes or topics) mentioned in the following bodywash review. Provide them as a comma-separated list.",
        "The smell is wonderful but it costs too much money.",
        "", 
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print("Model Prediction:", tokenizer.batch_decode(outputs)[0].split("### Response:\n")[-1])

In [ ]:
# 7. Save and Zip the Result
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

!zip -r lora_model.zip lora_model
#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
print("--------------------------------------------------")
print("DONE! Download 'lora_model.zip' from the files menu on the left.")
print("--------------------------------------------------")